In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, SimpleRNN
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import numpy as np
import pickle

# Load the dataset
df = pd.read_parquet('train-00000-of-00001.parquet')
df_test = pd.read_parquet('test-00000-of-00001.parquet')

# Display the first few rows of the dataframe
print(df.head())
print(df_test.head())

# Define label encoding
label_encoding = {"B-O": 0, "B-AC": 1, "B-LF": 2, "I-LF": 3}

# Convert the ner_tags to numerical labels
train_label = [[label_encoding[tag] for tag in sample] for sample in df["ner_tags"]]
test_label = [[label_encoding[tag] for tag in sample] for sample in df_test["ner_tags"]]

# Extract text data for TF-IDF encoding
X_train = [' '.join(tokens) for tokens in df['tokens']]
X_test = [' '.join(tokens) for tokens in df_test['tokens']]

# TF-IDF encoding
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Prepare sequences for RNN
X_train_tfidf_seq = [tfidf_vectorizer.transform(tokens).toarray().reshape(-1, 1, 5000) for tokens in df['tokens']]
X_test_tfidf_seq = [tfidf_vectorizer.transform(tokens).toarray().reshape(-1, 1, 5000) for tokens in df_test['tokens']]

# Flatten the sequences and corresponding labels
X_train_tfidf_seq_flat = np.vstack(X_train_tfidf_seq)
y_train_flat = np.concatenate(train_label)

# Define the RNN model
model = Sequential([
    SimpleRNN(128, input_shape=(None, 5000), activation='relu'),
    Dense(256, activation='relu'),
    Dense(len(label_encoding), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train_tfidf_seq_flat, y_train_flat, epochs=10, batch_size=32, verbose=1)

# Flatten the test sequences and corresponding labels
X_test_tfidf_seq_flat = np.vstack(X_test_tfidf_seq)
y_test_flat = np.concatenate(test_label)

# Evaluate the model on the test set
y_pred = model.predict(X_test_tfidf_seq_flat)
y_pred_classes = np.argmax(y_pred, axis=1)

# Calculate and print confusion matrix
conf_matrix = confusion_matrix(y_test_flat, y_pred_classes)
print("Confusion Matrix:")
print(conf_matrix)

# Print classification report
print(classification_report(y_test_flat, y_pred_classes, target_names=label_encoding.keys()))

# Calculate and print accuracy
accuracy = accuracy_score(y_test_flat, y_pred_classes)
print("Accuracy:", accuracy)

# Save the model
model.save('rnn_model.h5')

# Save the TF-IDF vectorizer
with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf_vectorizer, f)

# Save the label encoding
with open('label_encoding.pkl', 'wb') as f:
    pickle.dump(label_encoding, f)


                                              tokens  \
0  [For, this, purpose, the, Gothenburg, Young, P...   
1  [The, following, physiological, traits, were, ...   
2  [Minor, H, antigen, alloimmune, responses, rea...   
3                 [EPI, =, Echo, planar, imaging, .]   
4  [Furthermore, ,, eNOS, -, derived, NO, S, -, n...   

                                            pos_tags  \
0  [ADP, DET, NOUN, DET, PROPN, PROPN, PROPN, PRO...   
1  [DET, ADJ, ADJ, NOUN, AUX, VERB, PUNCT, ADJ, N...   
2  [ADJ, PROPN, NOUN, ADJ, NOUN, ADV, VERB, ADP, ...   
3            [PROPN, PUNCT, NOUN, NOUN, NOUN, PUNCT]   
4  [ADV, PUNCT, PROPN, PUNCT, VERB, PROPN, NOUN, ...   

                                            ner_tags  
0  [B-O, B-O, B-O, B-O, B-LF, I-LF, I-LF, I-LF, I...  
1  [B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-LF, I-LF...  
2  [B-O, B-AC, B-O, B-O, B-O, B-O, B-O, B-O, B-O,...  
3                 [B-AC, B-O, B-LF, I-LF, I-LF, B-O]  
4  [B-O, B-O, B-AC, B-O, B-O, B-AC, B-O, B-O, B-O..

C:\Users\sheet\anaconda3\DataScience\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.8246 - loss: 0.6597
Epoch 2/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.8642 - loss: 0.4201
Epoch 3/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.8764 - loss: 0.3713
Epoch 4/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.8811 - loss: 0.3516
Epoch 5/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.8784 - loss: 0.3554
Epoch 6/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.8820 - loss: 0.3479
Epoch 7/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.8800 - loss: 0.3466
Epoch 8/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.8803 - loss: 0.3467
Epoch 9/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.8801 - loss: 0.3447
Epoch 10/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.8817 - loss: 0.3364
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


Confusion Matrix:
[[4152   22   23   95]
 [ 202   64    0    4]
 [ 122    0   18   10]
 [ 205    0   14   69]]
              precision    recall  f1-score   support

         B-O       0.89      0.97      0.93      4292
        B-AC       0.74      0.24      0.36       270
        B-LF       0.33      0.12      0.18       150
        I-LF       0.39      0.24      0.30       288

    accuracy                           0.86      5000
   macro avg       0.59      0.39      0.44      5000
weighted avg       0.83      0.86      0.84      5000

Accuracy: 0.8606


In [ ]:
from flask import Flask, request, jsonify
import numpy as np
import pickle
from tensorflow.keras.models import load_model
import logging
from datetime import datetime

# Load the model and preprocessing artifacts
model = load_model('rnn_model.h5')

with open('tfidf_vectorizer.pkl', 'rb') as f:
    tfidf_vectorizer = pickle.load(f)

with open('label_encoding.pkl', 'rb') as f:
    label_encoding = pickle.load(f)

# Create a reverse label encoding map for decoding predictions
reverse_label_encoding = {v: k for k, v in label_encoding.items()}

# Configure logging
logging.basicConfig(filename='model_predictions.log', level=logging.INFO)

app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    data = request.json
    tokens = data['tokens']

    # Preprocess the input text for each token separately
    token_embeddings = tfidf_vectorizer.transform(tokens).toarray().reshape(len(tokens), 1, -1)
    
    # Predict using the RNN model
    y_pred = model.predict(token_embeddings)
    y_pred_classes = np.argmax(y_pred, axis=-1)

    # Decode the predictions
    decoded_predictions = [reverse_label_encoding[i] for i in y_pred_classes]

    # Log the request and predictions
    log_entry = {
        'timestamp': datetime.now().isoformat(),
        'tokens': tokens,
        'predictions': decoded_predictions
    }
    logging.info(log_entry)

    return jsonify({
        'prediction': decoded_predictions
    })

if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step


INFO:werkzeug:127.0.0.1 - - [23/May/2024 21:47:00] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/May/2024 21:48:00] "POST /predict HTTP/1.1" 400 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step


...

INFO:werkzeug:127.0.0.1 - - [23/May/2024 22:41:09] "POST /predict HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step


INFO:werkzeug:127.0.0.1 - - [23/May/2024 22:41:09] "POST /predict HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


INFO:werkzeug:127.0.0.1 - - [23/May/2024 22:41:10] "POST /predict HTTP/1.1" 200 -
